In [15]:
import pandas as pd
df = pd.read_csv('gemstone.csv')

df.drop(labels='id',axis=1,inplace=True)

X = df.drop(labels='price',axis=1)
y = df['price']

In [16]:

category_cols = X.columns[X.dtypes==object]
numerical_cols = X.columns[X.dtypes!=object]
print(category_cols)
print(numerical_cols)

Index(['cut', 'color', 'clarity'], dtype='object')
Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


In [17]:
# Giving ranks to each categorical feature
cut_map = {1:'Fair',2:'Good',3:'Very Good',4:'Premium',5:'Ideal'}
clarity_map = {"T1":1,"ST2":2,"ST1":3,"VS2":4,"VS1":5,"VVS2":6,"VVS1":7,"I1":8}
color_map = {"D":1,"E":2,"F":3,"G":4,"H":5,"I":6}

In [18]:
df['cut'] = df['cut'].map(cut_map)
df['clarity'] = df['clarity'].map(clarity_map)
df['color'] = df['color'].map(color_map)

In [19]:
# when doing with pipeline following procedures would be done. Arrange in rank order.
cut_cateogry = ['Fair','Good','Very Good','Premium','Ideal']
clarity_category = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
color_category = ['D','E','F','G','H','I','J']

In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [21]:
#Numerical Pipeline
numerical_pipeline = Pipeline(
                    steps=[('imputer',SimpleImputer(strategy ='median')),
                          ('scaler',StandardScaler())]
                        )

#Categorical Pipeline
categorical_pipeline = Pipeline(
                        steps=[('imputer',SimpleImputer(strategy ='most_frequent')),
                            ('ordinalencoder',OrdinalEncoder(categories=[cut_cateogry,color_category,clarity_category])),
                            ('scaler',StandardScaler())
                                ]
)

preprocessor = ColumnTransformer([
                        ('numerical_pipeline',numerical_pipeline,numerical_cols),
                        ('categorical_pipeline',categorical_pipeline,category_cols)]
)

In [22]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3,random_state=33)

In [23]:
xtrain = pd.DataFrame(preprocessor.fit_transform(xtrain),columns=preprocessor.get_feature_names_out())
xtest = pd.DataFrame(preprocessor.transform(xtest),columns=preprocessor.get_feature_names_out())

In [24]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [25]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [26]:
## Train multiple models
## Model Ecaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(xtrain,ytrain)

    #Make Predictions
    y_pred=model.predict(xtest)

    mae, rmse, r2_square=evaluate_model(ytest,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1003.9683231417791
MAE: 673.4979988107968
R2 score 93.75571899255426


Lasso
Model Training Performance
RMSE: 1004.1774865202909
MAE: 674.7072899734009
R2 score 93.75311689658488


Ridge
Model Training Performance
RMSE: 1003.9723371874084
MAE: 673.5296527449865
R2 score 93.75566906094106


Elasticnet
Model Training Performance
RMSE: 1526.4076959180634
MAE: 1060.7705550659125
R2 score 85.56610596968004


